### Ollama - llama test

In [ ]:
from langchain_ollama import ChatOllama

llm = ChatOllama(model="llama3.2:1b")

### Str Output Parser

In [ ]:
# 원래 답변 : 아래와 같이 지저분함
# AIMessage(content='The capital of France is Paris.', additional_kwargs={}, response_metadata={'model': 'llama3.2:1b', 'created_at': '2025-10-28T01:43:06.1172334Z', 'done': True, 'done_reason': 'stop', 'total_duration': 4064044000, 'load_duration': 3259412400, 'prompt_eval_count': 32, 'prompt_eval_duration': 479329800, 'eval_count': 8, 'eval_duration': 301487700, 'model_name': 'llama3.2:1b', 'model_provider': 'ollama'}, id='lc_run--51ac0589-2b94-4bfa-a72a-fb36edf96204-0', usage_metadata={'input_tokens': 32, 'output_tokens': 8, 'total_tokens': 40})

from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

prompt_template = PromptTemplate(
    template= "What is the capital of {country}? Return only the name of the capital.",
    input_variables=["country"]
)

prompt = prompt_template.invoke({"country": "France"})

print(prompt)

ai_message = llm.invoke(prompt)

output_parser = StrOutputParser()

answer = output_parser.invoke(llm.invoke(prompt))

In [ ]:
ai_message

In [ ]:
answer

### JsonOutputParser 는 잘 안됨

In [ ]:
from langchain_core.output_parsers import JsonOutputParser

country_detail_prompt = PromptTemplate(
    template="""Give following information about {country}:
    - Capital
    - Population
    - Languages
    - Currency

    return it in JSON format. and return the JSON dictionary only
    """,
    input_variables=["country"],
)

country_detail_prompt.invoke({"country": "France"})

output_parser = JsonOutputParser()
json_ai_message = llm.invoke(country_detail_prompt)
answer = output_parser.invoke(json_ai_message)

In [ ]:
# 문제 있음 (```이 언제 어디에 위치할지 모름)
json_ai_message.content

### pydantic 을 사용해 구조화된 응답 출력 가능

In [ ]:
# pydantic 의 BaseModel을 사용하여 구조화된 출력을 생성

from pydantic import BaseModel, Field

class CountryDetail(BaseModel):
    capital: str = Field(description="The capital of the country")
    population: int = Field(description="The population of the country")
    languages: list[str] = Field(description="The languages spoken in the country")
    currency: str = Field(description="The currency of the country")

structued_llm = llm.with_structured_output(CountryDetail)
json_ai_message = structued_llm.invoke(country_detail_prompt.invoke({"country": "France"}))

print(structued_llm)

In [ ]:
json_ai_message

In [ ]:
print(json_ai_message.capital)
print(json_ai_message.population)
print(json_ai_message.languages)
print(json_ai_message.currency)

json_ai_message.model_dump()
